In [2]:
import sys
sys.path.append('/home/gridsan/vyuan/.local/lib/python3.9/site-packages/')
sys.path.append("../src")

In [3]:
from utils import data_utils
from utils import conceptset_utils

In [4]:
from pathlib import Path
root = Path.cwd().parent.parent

import os
os.chdir(root)

In [5]:
"""
ConceptNet params:
LIMIT:how many relations to look up, higher limit -> larger concept set
RELATIONS: which relations to include in search 

filters:
CLASS_SIM_CUTOFF: Concenpts with cos similarity higher than this to any class will be removed
OTHER_SIM_CUTOFF: Concenpts with cos similarity higher than this to another concept will be removed
MAX_LEN: max number of characters in a concept

PRINT_PROB: what percentage of filtered concepts will be printed

"""

LIMIT = 200
RELATIONS = ["HasA", "IsA", "PartOf", "HasProperty", "MadeOf"]#, "AtLocation"]

CLASS_SIM_CUTOFF = 0.85
OTHER_SIM_CUTOFF = 0.9 
MAX_LEN = 30

PRINT_PROB = 0.2

dataset = "doctor_nurse"
save_name = 'data/concept_sets/conceptnet/conceptnet_{}_filtered_new.txt'.format(dataset)

In [6]:
cls_file = data_utils.LABEL_FILES[dataset]

with open(cls_file, 'r') as f:
    classes = f.read().split('\n')

# Collect initial concepts

In [13]:
concepts = conceptset_utils.get_init_conceptnet(classes, LIMIT, RELATIONS)
# Store the concept in a file, to use it with the GPU kernel
with open(save_name, 'w') as f:
    for concept in concepts:
        f.write(concept + '\n')

100%|██████████| 2/2 [00:00<00:00,  4.90it/s]


In [7]:
# Load the concepts from save_name in the variable concepts
with open(save_name, 'r') as f:
    concepts = f.read().split('\n')

concepts = conceptset_utils.remove_too_long(concepts, MAX_LEN, PRINT_PROB)

8 8


# Filter out concepts too similar to class names

In [10]:
concepts = conceptset_utils.filter_too_similar_to_cls(concepts, classes, CLASS_SIM_CUTOFF, print_prob=PRINT_PROB)

7
Class:nurse - Deleting nurse
4


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

# Filter out concepts too similar to each other

In [ ]:
concepts = conceptset_utils.filter_too_similar(concepts, OTHER_SIM_CUTOFF, print_prob=PRINT_PROB)

In [ ]:
with open(save_name, "w") as f:
    f.write(concepts[0])
    for concept in concepts[1:]:
        f.write("\n" + concept)